**note**: this is example for using NVHelp's previous LLM as a judge prompt that evaluate the answer from 4 additional aspects: conciseness, empathy, helpfulness, correctness

In [1]:
import os
import pandas as pd
from tqdm import tqdm
import json
import copy
import random
import string
from datetime import datetime

from utils import *

In [2]:
assert os.getcwd().endswith("user-guide"), "ensure the notebook is started in the user-guide directory"

In [3]:
# example using the small dataset
excell_fn = "template/nvbot_evaluation_complex.xlsx"
judge_prompt_fn= "template/prompts/nvbot.jsonl"

output_dir = "dataset_output"

### Build Questions

In [4]:
df = pd.read_excel(excell_fn)
print(df.columns)
questions = build_questions(df)
write_questions(output_dir=output_dir, questions=questions)

Index(['ID', 'question', 'correct_answer', 'required_citations', 'helpfulness',
       'empathy', 'short_ans'],
      dtype='object')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1850.32it/s]

Total questions loaded:  5
Questions file written to: dataset_output/question.jsonl


### Build References

In [5]:
def write_references(output_dir, references: list = []):
    fn = "reference.jsonl"
    ref_dir = create_folder_if_not_exists(output_dir + "/reference_answer")
    jsonl_writer(references, os.path.join(ref_dir, fn))

In [6]:
# Create reference folder
references = build_references(df)
write_references(output_dir=output_dir, references=references)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 3876.44it/s]


### Validate Judge Prompts
**Note**: look trough `template/prompts/*` for examples of prompts.

In [21]:
### validate, else bootstrap from default judge promtpts
DEFAULT_JUDGE_PROMPT_JSONL = "template/prompts/nvbot.jsonl"
validate_judge_prompts(output_dir, DEFAULT_JUDGE_PROMPT_JSONL, df)


dataset_output/judge_prompts.jsonl
Prompt variables: ['helpfulness', 'empathy', 'short_ans', 'question', 'correct_answer', 'required_citations', 'answer']
Columns in the dataset: ['ID', 'question', 'correct_answer', 'required_citations', 'helpfulness', 'empathy', 'short_ans']


In [22]:
### Prepare judge_prompt_parameters folders
judge_params_dir = f"{output_dir}/judge_prompt_parameters"
create_folder_if_not_exists(judge_params_dir)

'dataset_output/judge_prompt_parameters'

In [33]:
create_judge_prompt_parameters(judge_params_dir, df)